In [2]:
from custom_clss_and_fncs import * 
import torch
import torch_geometric
from datetime import datetime
device = 'cpu'
if torch.cuda.is_available():
    torch.cuda.init()
    if torch.cuda.is_initialized():
        device = 'cuda:0'
#device = torch.device(device)
print(f"{device = }")




device = 'cuda:0'


In [ ]:
#model
class regresor_GIN_v3(torch.nn.Module):
    
    def __init__(
        self, 
        target_node_idx: int,
        n_nodes : int, 
        num_features : int, 
        out_channels: int,
        dropout : float = 0.09, 
        hidden_dim : int = 5, 
        #heads : int = 5,
        LeakyReLU_slope : float = 0.01,

        num_layers: int = 3
    ):
        super(regresor_GIN_v3, self).__init__() # TODO: why SUPER gato? 
        self.n_nodes = n_nodes
        self.dropout = dropout
        self.num_features = num_features
        self.target_node_idx = target_node_idx
        self.out_channels = out_channels
        
        self.GIN_layers =  GIN(in_channels= num_features, hidden_channels= hidden_dim, num_layers= num_layers, 
                               out_channels= out_channels, dropout=dropout,  jk=None, act='LeakyReLU', act_first = True)#.to('cuda')        
        self.FC1        = Linear(in_features=out_channels, out_features=1, bias=True)#.to('cuda')
        #self.FC2        = Linear(in_features=n_nodes, out_features=1, bias=True)#.to('cuda')        #self.leakyrelu = LeakyReLU(LeakyReLU_slope).to('cuda')
        self.leakyrelu = LeakyReLU(LeakyReLU_slope)#.to('cuda')
    def forward(self, x):
        data       = x.x 
        edge_index = x.edge_index
        batch_size = x.y.shape[0]

        x     = self.GIN_layers(data, edge_index)
        x     = x.reshape(batch_size, self.n_nodes, self.out_channels)
        x     = self.FC1(self.leakyrelu(x))
        
        return  x[:,self.target_node_idx,:].squeeze()



In [ ]:
def train_one_epoch(modelo: regresor_GIN_v3,
                    optimizer, 
                    train_loader: torch_geometric.loader.dataloader.DataLoader,
                    loss_fun: torch.nn.modules.loss,
                    scaler:torch.cuda.amp.grad_scaler.GradScaler,
                    swa_start:int,
                    swa_model,swa_scheduler,scheduler, 
                    n_batches_report:int, device:str='cpu' ):
    running_loss = 0.
    last_loss = 0.
    for i, data in enumerate(train_loader):
        assert not data.is_cuda   
        if (device == 'cuda:0') | (device == 'cuda'):                            
            data.to(device, non_blocking=True) 
            assert data.is_cuda
        
                
        optimizer.zero_grad(set_to_none=True) # Zero your gradients for every batch
        
        if (device == 'cuda:0') | (device == 'cuda'):
            with torch.cuda.amp.autocast():      
                predictions = modelo(data)# Make predictions for this batch
                loss        = loss_fun(predictions, data.y)
            scaler.scale(loss).backward()

            if (i+1) % 2 == 0:  
                if i > swa_start:
                    swa_model.update_parameters(modelo)
                    swa_scheduler.step()
                else:
                    scheduler.step()
            
            #check_seen_y.extend(data.y.squeeze().tolist())
            #loss.backward()  # Derive gradients.
                scaler.step(optimizer)
                scaler.update()
            #optimizer.step()  # Update parameters based on gradients.
                running_loss += loss.item()
                 
            
        else:
            with torch.cuda.amp.autocast():
                predictions = modelo(data)# Make predictions for this batch
                loss        = loss_fun(predictions, data.y)
            loss.backward()
            if (i+1) % 2 == 0:  
                if i > swa_start:
                    swa_model.update_parameters(modelo)
                    swa_scheduler.step()
                else:
                    scheduler.step()
                optimizer.step()
                running_loss += loss.item()   
            
        
    

            
        #It reports on the loss for every 100 batches.
        if i % n_batches_report == 99:
            last_loss = running_loss / n_batches_report
            running_loss = 0.
    torch.optim.swa_utils.update_bn(train_loader, swa_model, device=device)
    return last_loss
